In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from math import ceil

In [3]:
# Funciones que se usarán después

# Sacar los números de las cadenas y el promedio de estos, para tener valores más aproximados
def cadenas_to_numero(cadena):
    if isinstance(cadena, str):
        numeros = re.findall(r'\d+\.*\d*', cadena)
        if len(numeros) == 1:
            return float(numeros[0])
        elif len(numeros) == 2:
            numeros_float = [float(numero) for numero in numeros]
            return sum(numeros_float) / 2
    return None

# Eliminar la palabra estrato, para manejar este como variable numérica
def eliminar_palabra_estrato(cadena):
    if isinstance(cadena, str):
        numero_estrato = re.sub(r'Estrato\s*', '', cadena)
        if numero_estrato.isdigit():  # Comprobar si la cadena es un número
            return int(numero_estrato.strip())
    return None

# Convertir columnas categóricas a columnas numéricas
def map_departamento(departamento):
    if departamento == 'AMAZONAS':
        return 1
    elif departamento == 'ANTIOQUIA':
        return 2
    elif departamento == 'ARAUCA':
        return 3
    elif departamento == 'ATLANTICO':
        return 4
    elif departamento == 'BOGOTÁ':
        return 5
    elif departamento == 'BOLIVAR':
        return 6
    elif departamento == 'BOYACA':
        return 7
    elif departamento == 'CALDAS':
        return 8
    elif departamento == 'CAQUETA':
        return 9
    elif departamento == 'CASANARE':
        return 10
    elif departamento == 'CAUCA':
        return 12
    elif departamento == 'CESAR':
        return 13
    elif departamento == 'CHOCO':
        return 14
    elif departamento == 'CORDOBA':
        return 15
    elif departamento == 'CUNDINAMARCA':
        return 16
    elif departamento == 'GUAVIARE':
        return 17
    elif departamento == 'HUILA':
        return 18
    elif departamento == 'LA GUAJIRA':
        return 19
    elif departamento == 'MAGDALENA':
        return 20
    elif departamento == 'META':
        return 21
    elif departamento == 'NARIÑO':
        return 22
    elif departamento == 'NORTE SANTANDER':
        return 23
    elif departamento == 'PUTUMAYO':
        return 24
    elif departamento == 'QUINDIO':
        return 25
    elif departamento == 'RISARALDA':
        return 26
    elif departamento == 'SAN ANDRES':
        return 27
    elif departamento == 'SANTANDER':
        return 28
    elif departamento == 'SUCRE':
        return 29
    elif departamento == 'TOLIMA':
        return 30
    elif departamento == 'VALLE':
        return 31
    elif departamento == 'VAUPES':
        return 32
    else:
        return 0

def map_booleano(booleano):
    if booleano == 'Si' or booleano == 'S':
        return 1
    else:
        return 0

def map_educacion(educacion):
    if educacion == 'Educación profesional completa':
        return 1
    elif educacion == 'Educación profesional incompleta':
        return 2
    elif educacion == 'Postgrado':
        return 3
    elif educacion == 'Primaria completa':
        return 4
    elif educacion == 'Primaria incompleta':
        return 5
    elif educacion == 'Secundaria (Bachillerato) completa':
        return 6
    elif educacion == 'Secundaria (Bachillerato) incompleta':
        return 7
    elif educacion == 'Técnica o tecnológica completa':
        return 8
    elif educacion == 'Técnica o tecnológica incompleta':
        return 9
    else:
        return 0

def map_rendimiento(rendimiento):
    if rendimiento == 'alto':
        return 1
    elif rendimiento == 'bajo':
        return 2
    elif rendimiento == 'medio-alto':
        return 3
    elif rendimiento == 'medio-bajo':
        return 4
    else:
        return 0

In [4]:
# Cargar el archivo
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO/train.csv')
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


In [5]:
# Imprimir información de las columnas
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 17 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_TIENELAVADORA              652727 non-null  object
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object
 12  ESTU_PAGOMATRICULAPROPIO      

In [6]:
# Eliminar columna repetida
train = train.drop('FAMI_TIENEINTERNET.1',axis=1)

In [7]:
# Localizar las variables con valores faltantes
n = train.isna().sum()
n[n!=0]

ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_EDUCACIONMADRE               23664
dtype: int64

**TRATAMIENTO DE VARIABLES CATEGÓRICAS**

- ESTU_PRGM_ACADEMICO

In [8]:
# Imprimir datos de 'ESTU_PRGM_ACADEMICO', para saber qué correcciones se deben realizar
ESTU_PRGM_ACADEMICO = np.unique(train['ESTU_PRGM_ACADEMICO'])
ESTU_PRGM_ACADEMICO

array(['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES',
       'ACTIVIDAD FISICA Y DEPORTE', 'ACUICULTURA', 'ADMINISTRACION',
       'ADMINISTRACION  FINANCIERA', 'ADMINISTRACION & SERVICIO',
       'ADMINISTRACION AERONAUTICA', 'ADMINISTRACION AGROPECUARIA',
       'ADMINISTRACION AMBIENTAL',
       'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES',
       'ADMINISTRACION BANCARIA Y FINANCIERA', 'ADMINISTRACION COMERCIAL',
       'ADMINISTRACION COMERCIAL Y DE MERCADEO',
       'ADMINISTRACION DE COMERCIO EXTERIOR',
       'ADMINISTRACION DE EMPRESAS',
       'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL',
       'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES',
       'ADMINISTRACION DE EMPRESAS AGROPECUARIAS',
       'ADMINISTRACION DE EMPRESAS COMERCIALES',
       'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES',
       'ADMINISTRACION DE EMPRESAS TURISTICA',
       'ADMINISTRACION DE EMPRESAS TURISTICAS',
       'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS',
       

In [9]:
# Eliminar caracteres especiales, espacios, tildes, etc.
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].str.replace('?', '').str.replace('¿', '').str.replace('     ', ' ').str.replace('    ', ' ').str.replace('   ', ' ').str.replace('  ', ' ')
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [10]:
# Corregir errores tipográficos de forma manual
reemplazos_manuales = {
       '3 CICLO PROFESIONAL NEGOCIOS INTERNACIONALES': 'TERCER CICLO PROFESIONAL DE NEGOCIOS INTERNACIONALES',
       'ADMINISTRACIN DE EMPRESAS': 'ADMINISTRACION DE EMPRESAS',
       'ADMINISTRACIN LOGSTICA': 'ADMINISTRACION LOGISTICA',
       'ADMINISTRACIN PBLICA': 'ADMINISTRACION PUBLICA',
       'ADMINISTRACION & SERVICIO': 'ADMINISTRACION Y SERVICIO',
       'ADMINISTRACION DE EMPRESAS - CICLO PROFESIONAL': 'ADMINISTRACION DE EMPRESAS: CICLO PROFESIONAL',
       'ADMINISTRACION DE EMPRESAS TURISTICA' : 'ADMINISTRACION DE EMPRESAS TURISTICAS',
       'ADMINISTRACION DE EMPRESAS TURISTICAS POR CICLOS - CICLO PROFESIONAL': 'ADMINISTRACION DE EMPRESAS TURISTICAS POR CICLOS: CICLO PROFESIONAL',
       'ADMINISTRACION DE GESTISN HUMANA': 'ADMINISTRACION DE GESTION HUMANA',
       'ADMINSITRACION DE EMPRESAS': 'ADMINISTRACION DE EMPRESAS',
       'ADMINSITRACION TURISTICA Y HOTELERA': 'ADMINISTRACION TURISTICA Y HOTELERA',
       'CIENCIA DE LA INFORMACION - BIBLIOTECOLOGIA': 'CIENCIA DE LA INFORMACION Y BIBLIOTECOLOGIA',
       'COMUNICACIN SOCIAL': 'COMUNICACION SOCIAL',
       'COMUNICACIN SOCIAL Y PERIODISMO': 'COMUNICACION SOCIAL Y PERIODISMO',
       'COMUNICACIN VISUAL': 'COMUNICACION VISUAL',
       'COMUNICACION SOCIAL - PERIODISMO': 'COMUNICACION SOCIAL Y PERIODISMO',
       'COMUNICACION SOCIAL PERIODISMO': 'COMUNICACION SOCIAL Y PERIODISMO',
       'COMUNICACION SOCIAL- PERIODISMO': 'COMUNICACION SOCIAL Y PERIODISMO',
       'COMUNICACION SOCIAL-PERIODISMO': 'COMUNICACION SOCIAL Y PERIODISMO',
       'COMUNICACION SOCIALY PERIODISMO': 'COMUNICACION SOCIAL Y PERIODISMO',
       'CONTADURIA PBLICA': 'CONTADURIA PUBLICA',
       'CONTADURIA PUBLICA.': 'CONTADURIA PUBLICA',
       'DEPORTE Y ACTIVIDADA FISICA': 'DEPORTE Y ACTIVIDAD FISICA',
       'DISENO DE ESPACIOS - ESCENARIO': 'DISENO DE ESPACIOS Y ESCENARIOS',
       'DISENO DE MODA': 'DISENO DE MODAS',
       'DISEO CROSSMEDIA': 'DISENO CROSSMEDIA',
       'DISEO DE COMUNICACIN GRFICA': 'DISENO DE COMUNICACION GRAFICA',
       'DISEO Y ADMINISTRACIN DE NEGOCIOS DE LA MODA': 'DISENO Y ADMINISTRACION DE NEGOCIOS DE LA MODA',
       'ECONOMA': 'ECONOMIA',
       'GEOLOGA': 'GEOLOGIA',
       'GESTIN DEPORTIVA': 'GESTION DEPORTIVA',
       'INGENIERA DE SISTEMAS': 'INGENIERIA DE SISTEMAS',
       'INGENIERA ELCTRICA': 'INGENIERIA ELECTRICA',
       'INGENIERA EN SOFTWARE': 'INGENIERIA EN SOFTWARE',
       'INGENIERA INDUSTRIAL': 'INGENIERIA INDUSTRIAL',
       'INTRUMENTACION QUIRURGICA': 'INSTRUMENTACION QUIRURGICA',
       'LETRAS - FILOLOGIA HISPANICA': 'LETRAS Y FILOLOGIA HISPANICA',
       'LICENCIATURA EN ARTE Y FOLKLORE Y CULTURA': 'LICENCIATURA EN ARTE, FOLKLORE Y CULTURA',
       'LICENCIATURA EN ARTES ESCNICAS': 'LICENCIATURA EN ARTES ESCENICAS',
       'LICENCIATURA EN CIENCIAS NATURALES: FISICA QUIMICA Y BIOLOGIA': 'LICENCIATURA EN CIENCIAS NATURALES: FISICA, QUIMICA Y BIOLOGIA',
       'LICENCIATURA EN EDUCACIN ARTSTICA': 'LICENCIATURA EN EDUCACION ARTISTICA',
       'LICENCIATURA EN EDUCACIN BSICA PRIMARIA': 'LICENCIATURA EN EDUCACION BASICA PRIMARIA',
       'LICENCIATURA EN EDUCACIN CON NFASIS EN CIENCIAS SOCIALES Y AMBIENTALES': 'LICENCIATURA EN EDUCACION CON ENFASIS EN CIENCIAS SOCIALES Y AMBIENTALES',
       'LICENCIATURA EN EDUCACIN FSICA RECREACIN Y DEPORTES': 'LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACIN INFANTIL': 'LICENCIATURA EN EDUCACION INFANTIL',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION ARTISTICA-MUSICA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION ARTISTICA Y MUSICA',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA, DEPORTES Y RECREACION': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTE': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA,RECREACION Y DEPORTES': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES - LENGUA CASTELLANA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES LENGUA CASTELLANA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, LENGUA CASTELLANA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES,LENGUA CASTELLANA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES,LENGUA CASTELLANA E INGLES': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, LENGUA CASTELLANA E INGLES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES- LENGUA CASTELLANA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES-INGLES': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES E INGLES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: ESPANOL E INGLES': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, ESPANOL E INGLES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: LENGUA CASTELLANA E IDIOMA EXTRANJERO: INGLES': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, LENGUA CASTELLANA E INGLES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES: LENGUA CASTELLANA E INGLES': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES, LENGUA CASTELLANA E INGLES',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICA HUMANIDADES Y LENGUA CASTELLANA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN MATEMATICAS, HUMANIDADES Y LENGUA CASTELLANA',
       'LICENCIATURA EN EDUCACION BASICA EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
       'LICENCIATURA EN EDUCACION BASICA EN DANZA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN DANZAS',
       'LICENCIATURA EN EDUCACION BASICA ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
       'LICENCIATURA EN EDUCACION BASICA ENFASIS EN CIENCIAS SOCIALES': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS SOCIALES',
       'LICENCIATURA EN EDUCACION BASICA ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTE': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION BASICA PRIMARIA:ENFASIS EN CIENCIAS SOCIALES': 'LICENCIATURA EN EDUCACION BASICA PRIMARIA CON ENFASIS EN CIENCIAS SOCIALES',
       'LICENCIATURA EN EDUCACION BASICA, CON ENFASIS EN HUMANIDADES, LENGUA CASTELLANA': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES Y LENGUA CASTELLANA',
       'LICENCIATURA EN EDUCACION BASICA, ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL': 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN CIENCIAS NATURALES Y EDUCACION AMBIENTAL',
       'LICENCIATURA EN EDUCACION BILINGUE- ESPANOL E INGLES': 'LICENCIATURA EN EDUCACION BILINGUE: ESPANOL E INGLES',
       'LICENCIATURA EN EDUCACION BILINGUE-ESPANOL-INGLES': 'LICENCIATURA EN EDUCACION BILINGUE: ESPANOL E INGLES',
       'LICENCIATURA EN EDUCACION ENFASIS EN CIENCIAS SOCIALES': 'LICENCIATURA EN EDUCACION CON ENFASIS EN CIENCIAS SOCIALES',
       'LICENCIATURA EN EDUCACION ENFASIS EN INFORMATICA': 'LICENCIATURA EN EDUCACION CON ENFASIS EN INFORMATICA',
       'LICENCIATURA EN EDUCACION FISICA DEPORTES Y RECREACION': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTE': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION FISICA, DEPORTE Y RECREACION ENFASIS RURAL': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES CON ENFASIS RURAL',
       'LICENCIATURA EN EDUCACION FISICA, DEPORTES Y RECREACION': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTE': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION FISICA,RECREACION Y DEPORTE': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION FISICA,RECREACION Y DEPORTES': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN EDUCACION RELIGIOSA CON ENFASIS EN BIBLIA': 'LICENCIATURA EN EDUCACION RELIGIOSA CON ENFASIS EN LA BIBLIA',
       'LICENCIATURA EN EDUCACON FISICA, RECREACION Y DEPORTES': 'LICENCIATURA EN EDUCACION FISICA, RECREACION Y DEPORTES',
       'LICENCIATURA EN FILOSOFA Y HUMANIDADES': 'LICENCIATURA EN FILOSOFIA Y HUMANIDADES',
       'LICENCIATURA EN IDIOMA EXTRANJERO - INGLES': 'LICENCIATURA EN IDIOMA EXTRANJERO: INGLES',
       'LICENCIATURA EN IDIOMAS MODERNOS ESPANOL- INGLES': 'LICENCIATURA EN IDIOMAS MODERNOS: ESPANOL E INGLES',
       'LICENCIATURA EN INGLES - ESPANOL': 'LICENCIATURA EN INGLES Y ESPANOL',
       'LICENCIATURA EN INGLES-ESPANOL': 'LICENCIATURA EN INGLES Y ESPANOL',
       'LICENCIATURA EN LENGUA CASTELLANA,INGLES Y FRANCES': 'LICENCIATURA EN LENGUA CASTELLANA, INGLES Y FRANCES',
       'LICENCIATURA EN LENGUAS EXTRANJERAS CON NFASIS EN INGLS': 'LICENCIATURA EN LENGUAS EXTRANJERAS CON ENFASIS EN INGLES',
       'LICENCIATURA EN LENGUAS EXTRANJERAS INGLES - FRANCES': 'LICENCIATURA EN LENGUAS EXTRANJERAS: INGLES Y FRANCES',
       'LICENCIATURA EN LENGUAS EXTRANJERAS INGLES-FRANCES': 'LICENCIATURA EN LENGUAS EXTRANJERAS: INGLES Y FRANCES',
       'LICENCIATURA EN LENGUAS EXTRANJERAS: INGLES-FRANCES': 'LICENCIATURA EN LENGUAS EXTRANJERAS: INGLES Y FRANCES',
       'LICENCIATURA EN LENGUAS MODERNAS INGLES-FRANCES': 'LICENCIATURA EN LENGUAS MODERNAS: INGLES Y FRANCES',
       'LICENCIATURA EN LENGUAS MODERNAS, INGLES Y FRANCES': 'LICENCIATURA EN LENGUAS MODERNAS: INGLES Y FRANCES',
       'LICENCIATURA EN MATEMTICAS': 'LICENCIATURA EN MATEMATICAS',
       'LICENCIATURA EN PEDAGOGA INFANTIL': 'LICENCIATURA EN PEDAGOGIA INFANTIL',
       'LICENCIATURA LENGUAS EXTRANJERAS CON ENFASIS EN INGLES': 'LICENCIATURA EN LENGUAS EXTRANJERAS CON ENFASIS EN INGLES',
       'LICENCIATURA MSICA': 'LICENCIATURA EN MUSICA',
       'LICENCIATURA PEDAGOGIA INFANTIL': 'LICENCIATURA EN PEDAGOGIA INFANTIL',
       'Licenciatura en Lenguas Extranjeras con Enfasis en Ingles': 'LICENCIATURA EN LENGUAS EXTRANJERAS CON ENFASIS EN INGLES',
       'MATEMATICA APLICADA': 'MATEMATICAS APLICADAS',
       'MERCADEO PUBLICIDAD Y VENTAS': 'MERCADEO, PUBLICIDAD Y VENTAS',
       'MUSICA INSTRUMENTO': 'MUSICA INSTRUMENTAL',
       'MUSICA- CANTO': 'MUSICA Y CANTO',
       'PROFESIONAL EN COMUNICACION SOCIAL - PERIODISMO': 'PROFESIONAL EN COMUNICACION SOCIAL Y PERIODISMO',
       'PROFESIONAL EN GASTRONOMA': 'PROFESIONAL EN GASTRONOMIA',
       'PROFESIONAL UNIVERSITARIO DISENO GRAFICO DIGITAL': 'PROFESIONAL UNIVERSITARIO EN DISENO GRAFICO DIGITAL',
       'PSICOLOGA': 'PSICOLOGIA',
       'QUIMICA.': 'QUIMICA',
       'QUMICA FARMACUTICA': 'QUIMICA FARMACEUTICA',
       'TRADUCCION INGLES-FRANCES-ESPANOL': 'TRADUCCION INGLES, FRANCES Y ESPANOL'
}

# Realizar el reemplazo manual
train['ESTU_PRGM_ACADEMICO'].replace(reemplazos_manuales, inplace=True)

- ESTU_PRGM_DEPARTAMENTO

In [11]:
# Aplicar la función map_departamento a la columna 'ESTU_PRGM_DEPARTAMENTO'
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].apply(map_departamento)

- ESTU_VALORMATRICULAUNIVERSIDAD

In [12]:
# Aplicar la función cadenas_to_numero a la columna 'ESTU_VALORMATRICULAUNIVERSIDAD'
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(cadenas_to_numero)

- ESTU_HORASSEMANATRABAJA

In [13]:
# Aplicar la función cadenas_to_numero a la columna 'ESTU_HORASSEMANATRABAJA'
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].apply(cadenas_to_numero)

- FAMI_ESTRATOVIVIENDA

In [14]:
# Aplicar la función eliminar_palabra_estrato a la columna 'FAMI_ESTRATOVIVIENDA'
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].apply(eliminar_palabra_estrato)

- FAMI_TIENEINTERNET

In [15]:
# Aplicar la función map_booleano a la columna 'FAMI_TIENEINTERNET'
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].apply(map_booleano)

- FAMI_EDUCACIONPADRE

In [16]:
# Aplicar la función map_educacion a la columna 'FAMI_EDUCACIONPADRE'
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].apply(map_educacion)

- FAMI_TIENELAVADORA

In [17]:
# Aplicar la función map_booleano a la columna 'FAMI_TIENELAVADORA'
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].apply(map_booleano)

- FAMI_TIENEAUTOMOVIL

In [18]:
# Aplicar la función map_booleano a la columna 'FAMI_TIENEAUTOMOVIL'
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].apply(map_booleano)

- ESTU_PRIVADO_LIBERTAD

In [19]:
# Aplicar la función map_booleano a la columna 'ESTU_PRIVADO_LIBERTADL'
train['ESTU_PRIVADO_LIBERTAD'] = train['ESTU_PRIVADO_LIBERTAD'].apply(map_booleano)

- ESTU_PAGOMATRICULAPROPIO

In [20]:
# Aplicar la función map_booleano a la columna 'ESTU_PAGOMATRICULAPROPIO'
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].apply(map_booleano)

- FAMI_TIENECOMPUTADOR

In [21]:
# Aplicar la función map_booleano a la columna 'FAMI_TIENECOMPUTADOR'
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].apply(map_booleano)

- FAMI_EDUCACIONMADRE

In [22]:
# Aplicar la función map_educacion a la columna 'FAMI_EDUCACIONMADRE'
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].apply(map_educacion)

- RENDIMIENTO_GLOBAL

In [23]:
# Aplicar la función map_rendimiento a la columna 'RENDIMIENTO_GLOBAL'
train['RENDIMIENTO_GLOBAL'] = train['RENDIMIENTO_GLOBAL'].apply(map_rendimiento)

In [24]:
# Localizar las variables con valores faltantes
n = train.isna().sum()
n[n!=0]

ESTU_VALORMATRICULAUNIVERSIDAD    25815
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              35426
dtype: int64

In [25]:
# Asignarles valor 0
train.fillna(value=0,inplace=True)

In [26]:
# Convertir columnas a int
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].astype('int')
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].astype('int')
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].astype('int')

In [27]:
# Resultado final
train

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,5,6,10,3,1,9,1,1,0,0,1,3,3
1,645256,20212,DERECHO,4,3,0,3,0,8,1,0,0,0,1,9,2
2,308367,20203,MERCADEO Y PUBLICIDAD,5,3,30,3,1,6,1,0,0,0,0,6,2
3,470353,20195,ADMINISTRACION DE EMPRESAS,28,4,0,4,1,0,1,0,0,0,1,6,1
4,989032,20212,PSICOLOGIA,2,3,25,3,1,4,1,1,0,0,1,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,19,250,15,2,1,6,1,0,0,1,1,7,3
692496,754213,20212,PSICOLOGIA,23,3,30,3,1,5,1,0,0,0,1,7,2
692497,504185,20183,ADMINISTRACION EN SALUD OCUPACIONAL,5,1,10,3,1,6,1,0,0,1,1,7,4
692498,986620,20195,PSICOLOGIA,30,3,10,1,0,4,0,0,0,1,1,4,2


In [28]:
# Resumen estadístico
train.describe()

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,494606.130576,20198.366679,10.990553,88.635119,17.917184,2.404497,0.855616,4.476879,0.813560,0.341186,0.000049,0.438702,0.863061,4.869726,2.491769
std,285585.209455,10.535037,9.732601,168.027656,11.701883,1.217967,0.351479,2.614768,0.389462,0.474108,0.007007,0.496229,0.343783,2.573415,1.120126
min,1.000000,20183.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,247324.750000,20195.000000,5.000000,1.000000,10.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,1.000000
50%,494564.500000,20195.000000,5.000000,3.000000,15.000000,2.000000,1.000000,5.000000,1.000000,0.000000,0.000000,0.000000,1.000000,5.000000,2.000000
75%,741782.500000,20203.000000,19.000000,7.000000,30.000000,3.000000,1.000000,6.000000,1.000000,1.000000,0.000000,1.000000,1.000000,7.000000,3.000000
max,989286.000000,20213.000000,32.000000,500.000000,30.000000,6.000000,1.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000,4.000000


**ONE-HOT**

In [29]:
# Seleccionar solo las columnas categóricas
categorical_columns = train.select_dtypes(include=['object'])

# Aplicar one-hot encoding a las columnas categóricas
df_encoded = pd.get_dummies(train, columns=categorical_columns.columns).astype(int)

df_encoded

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,...,ESTU_PRGM_ACADEMICO_TERAPIA CARDIORRESPIRATORIA,ESTU_PRGM_ACADEMICO_TERAPIA OCUPACIONAL,ESTU_PRGM_ACADEMICO_TERAPIA RESPIRATORIA,ESTU_PRGM_ACADEMICO_TERAPIAS PSICOSOCIALES,ESTU_PRGM_ACADEMICO_TERCER CICLO PROFESIONAL DE NEGOCIOS INTERNACIONALES,ESTU_PRGM_ACADEMICO_TRABAJO SOCIAL,"ESTU_PRGM_ACADEMICO_TRADUCCION INGLES, FRANCES Y ESPANOL",ESTU_PRGM_ACADEMICO_TURISMO,ESTU_PRGM_ACADEMICO_URBANISMO,ESTU_PRGM_ACADEMICO_ZOOTECNIA
0,904256,20212,5,6,10,3,1,9,1,1,...,0,0,0,0,0,0,0,0,0,0
1,645256,20212,4,3,0,3,0,8,1,0,...,0,0,0,0,0,0,0,0,0,0
2,308367,20203,5,3,30,3,1,6,1,0,...,0,0,0,0,0,0,0,0,0,0
3,470353,20195,28,4,0,4,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,989032,20212,2,3,25,3,1,4,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,19,250,15,2,1,6,1,0,...,0,0,0,0,0,0,0,0,0,0
692496,754213,20212,23,3,30,3,1,5,1,0,...,0,0,0,0,0,0,0,0,0,0
692497,504185,20183,5,1,10,3,1,6,1,0,...,0,0,0,0,0,0,0,0,0,0
692498,986620,20195,30,3,10,1,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
